In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters
num_epochs = 10
batch_size = 4
learning_rate = 0.001

In [12]:
# dataset has PILImage images of range [0,1]
# We transform them to Tensors of normalized range [-1,1]

import torch.utils
import torch.utils.data


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




Files already downloaded and verified
Files already downloaded and verified


In [13]:
classes = ('plane', 'car', 'bird', 'cat','deer','dog', 'frog', 'horse', 'ship', 'truck')


In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [17]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%2000 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

print("Finished Training")

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        # max returns (value, index)
        _, predicted = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100.0 * n_correct/n_samples
    print(f'Accuracy of the network: {acc}%')

    for i in range(10):
        acc = 100.0 * n_class_correct[i]/n_class_samples[i]
        print(f"Accuracy of {classes[i]}: {acc}%")



Epoch [1/10], Step [2000/12500], Loss: 2.3139
Epoch [1/10], Step [4000/12500], Loss: 2.2494
Epoch [1/10], Step [6000/12500], Loss: 2.2183
Epoch [1/10], Step [8000/12500], Loss: 2.1923
Epoch [1/10], Step [10000/12500], Loss: 2.3259
Epoch [1/10], Step [12000/12500], Loss: 1.7966
Epoch [2/10], Step [2000/12500], Loss: 2.6780
Epoch [2/10], Step [4000/12500], Loss: 1.6778
Epoch [2/10], Step [6000/12500], Loss: 1.7714
Epoch [2/10], Step [8000/12500], Loss: 2.2453
Epoch [2/10], Step [10000/12500], Loss: 1.8089
Epoch [2/10], Step [12000/12500], Loss: 1.4431
Epoch [3/10], Step [2000/12500], Loss: 1.6185
Epoch [3/10], Step [4000/12500], Loss: 1.3743
Epoch [3/10], Step [6000/12500], Loss: 2.3259
Epoch [3/10], Step [8000/12500], Loss: 1.4128
Epoch [3/10], Step [10000/12500], Loss: 1.1711
Epoch [3/10], Step [12000/12500], Loss: 1.4646
Epoch [4/10], Step [2000/12500], Loss: 1.4231
Epoch [4/10], Step [4000/12500], Loss: 1.7287
Epoch [4/10], Step [6000/12500], Loss: 1.5126
Epoch [4/10], Step [8000/125